In [34]:
# -*- coding: utf-8 -*-
# ================================================================
# Part 2: Practical Challenges - A Robust Simulation Framework
#
# Implements:
# 1. Robust Best-Response Dynamics (BRD) for Proportional Sharing (PS)
#    - Features: Correct asynchronous random sweeps, multiple restarts,
#      in-round cycle detection, and robustness checks.
# 2. Correctly switches between Exact and Monte Carlo cost calculation.
#
# This is the final, corrected, self-contained script.
# ================================================================

from __future__ import annotations
import math
import time
import itertools
from pathlib import Path
from typing import Dict, Tuple, List, Any, Optional, Callable, Literal
from dataclasses import dataclass

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# ================================================================
# 1. Global Configuration & Utilities
# ================================================================

RESULTS_DIR = Path("results_practical")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
EPS = 1e-9  # Numerical comparison tolerance

@dataclass
class GameInstance:
    """A dataclass to hold all static parameters of a game instance."""
    n: int
    num_resources: int
    num_strategies_per_player: int
    weights: np.ndarray
    a_coeffs: np.ndarray
    d: int
    strategies: List[List[Tuple[int, ...]]]

class MasterRNG:
    """Master RNG for experiment-wide independence and reproducibility."""
    def __init__(self, seed: int = 42):
        self._rng = np.random.default_rng(seed)
    def spawn(self) -> np.random.Generator:
        child_seed = int(self._rng.integers(0, 2**31 - 1))
        return np.random.default_rng(child_seed)

# ================================================================
# 2. Core Game Logic: Instance Generation & Cost Calculation
# ================================================================

def generate_random_instance(
    n: int, num_strategies: int, num_resources: int, d: int, rng: np.random.Generator
) -> GameInstance:
    strategies = []
    resources = list(range(num_resources))
    for _ in range(n):
        cand = []

        # Force-inject two extreme strategies:
        # 1) single-edge; 2) full-cover
        single_edge = (int(rng.integers(0, num_resources)),)
        full_cover = tuple(range(num_resources))
        cand.append(single_edge)
        if num_resources > 1:
            cand.append(full_cover)

        # Fill the rest with random unique paths
        pool = set(cand)
        while len(pool) < num_strategies:
            k = int(rng.integers(1, num_resources + 1))
            path = tuple(sorted(rng.choice(resources, size=k, replace=False)))
            pool.add(path)

        strategies.append(list(pool)[:num_strategies])

    return GameInstance(
        n=n, num_resources=num_resources, num_strategies_per_player=num_strategies, d=d,
        weights=rng.uniform(0.5, 2.0, size=n),
        a_coeffs=rng.uniform(0.5, 1.5, size=num_resources),
        strategies=strategies
    )


def expected_costs_ps(
    instance: GameInstance, profile: Tuple[int, ...], p: float,
    mc_threshold_n: int = 12, mc_samples: int = 2000, rng: Optional[np.random.Generator] = None, types_bank=None
) -> Dict[str, Any]:
    """
    Calculates unconditional expected costs under the PS rule.
    Switches from exact enumeration to Monte Carlo for n > mc_threshold_n.
    """
    n, d, weights, a_coeffs = instance.n, instance.d, instance.weights, instance.a_coeffs
    paths = [instance.strategies[i][profile[i]] for i in range(n)]
    player_costs = np.zeros(n)

    if n <= mc_threshold_n:
        for mask in range(1 << n):
            k = bin(mask).count('1')
            prob = (p ** k) * ((1.0 - p) ** (n - k))
            if prob < 1e-16: continue
            
            active_players = {i for i in range(n) if (mask >> i) & 1}
            if not active_players: continue

            for e in range(instance.num_resources):
                active_on_edge = {i for i in active_players if e in paths[i]}
                if not active_on_edge: continue
                
                total_w = np.sum(weights[list(active_on_edge)])
                if total_w <= 0: continue
                
                edge_cost = a_coeffs[e] * (total_w ** d)
                for i in active_on_edge:
                    player_costs[i] += prob * edge_cost * (weights[i] / total_w)
        else:
            if types_bank is None:
                if rng is None: rng = np.random.default_rng()
                type_vectors = rng.random(size=(mc_samples, n)) < p
            else:
                type_vectors = types_bank  # CRN: reuse the same Bernoulli draws for all candidates

        for types in type_vectors:
            active_players = np.where(types)[0]
            if active_players.size == 0: continue
            
            for e in range(instance.num_resources):
                active_on_edge = [i for i in active_players if e in paths[i]]
                if not active_on_edge: continue

                total_w = np.sum(weights[active_on_edge])
                if total_w <= 0: continue
                
                edge_cost = a_coeffs[e] * (total_w ** d)
                for i in active_on_edge:
                    player_costs[i] += edge_cost * (weights[i] / total_w)
        player_costs /= len(type_vectors)

    return {"player_costs": player_costs, "social_cost": float(np.sum(player_costs))}

# ================================================================
# 3. Best Response Dynamics (BRD) Framework (Corrected)
# ================================================================

def _best_response_single_player(
    instance: GameInstance, profile: Tuple[int, ...], i: int, p: float,
    cost_fn: Callable, tie_break: str, noise_sigma: float, rng: np.random.Generator
) -> int:
    """Computes a best response for player i."""
    true_costs = np.array([
        cost_fn(instance, tuple(list(profile[:i]) + [s] + list(profile[i+1:])), p)['player_costs'][i]
        for s in range(instance.num_strategies_per_player)
    ])
    
    selection_costs = true_costs
    if noise_sigma > 0.0:
        selection_costs = true_costs + rng.normal(0.0, noise_sigma, size=true_costs.shape)
        
    min_cost = np.min(selection_costs)
    argmin_indices = np.where(np.abs(selection_costs - min_cost) <= EPS)[0]
    
    if tie_break == "first":
        return int(argmin_indices[0])
    else: # "random"
        return int(rng.choice(argmin_indices))

def run_brd_for_ps(
    instance: GameInstance, p: float,
    init_profile: Optional[Tuple[int, ...]] = None,
    max_rounds: int = 500, stable_rounds_needed: int = 3,
    tie_break: Literal["random", "first"] = "random",
    noise_sigma: float = 0.0, rng: Optional[np.random.Generator] = None,
    mc_threshold_n: int = 12, mc_samples: int = 2000
) -> Dict[str, Any]:
    """
    BRD for PS with asynchronous random sweeps and robust cycle detection.
    - Detect a cycle whenever a profile reappears at a different atomic update step.
    - Converged only if a full round makes no change, for 'stable_rounds_needed' consecutive rounds.
    """
    t0 = time.perf_counter()
    if rng is None:
        rng = np.random.default_rng()

    n, nstr = instance.n, instance.num_strategies_per_player

    types_bank = None
    if instance.n > mc_threshold_n:
        types_bank = (rng.random((mc_samples, instance.n)) < p)
    cost_fn = lambda inst, prof, p_val: expected_costs_ps(
        inst, prof, p_val,
        mc_threshold_n=mc_threshold_n,
        mc_samples=mc_samples,
        rng=rng,
        types_bank=types_bank
    )

    # Init profile
    profile = tuple(rng.integers(0, nstr, size=n)) if init_profile is None else tuple(init_profile)

    # ---- FIX: use atomic-step index for cycle detection ----
    # Map: profile -> last atomic step index when it was seen
    seen_at_step: Dict[Tuple[int, ...], int] = {profile: 0}
    atomic_steps = 0

    stable_streak = 0

    for round_num in range(1, max_rounds + 1):
        changed_this_round = False

        # Asynchronous (Gauss–Seidel): commit immediately
        for i in rng.permutation(n):
            best_s = _best_response_single_player(
                instance, profile, i, p, cost_fn, tie_break, noise_sigma, rng
            )

            # If no change, continue; still counts as an atomic decision
            atomic_steps += 1
            if best_s == profile[i]:
                continue

            # Apply change
            prof_list = list(profile)
            prof_list[i] = best_s
            profile = tuple(prof_list)
            changed_this_round = True

            # ---- FIX: in-round cycle detection using atomic step index ----
            # If we have seen this profile at any previous atomic step, it's a cycle.
            prev = seen_at_step.get(profile, None)
            if prev is not None and prev != atomic_steps:
                return {
                    "status": "CYCLE_DETECTED",
                    "rounds": round_num,
                    "iters": atomic_steps,
                    "time_sec": time.perf_counter() - t0,
                    "final_profile": profile
                }
            seen_at_step[profile] = atomic_steps

        # No player changed this round -> one stable round
        if not changed_this_round:
            stable_streak += 1
            if stable_streak >= stable_rounds_needed:
                return {
                    "status": "CONVERGED",
                    "rounds": round_num,
                    "iters": atomic_steps,
                    "time_sec": time.perf_counter() - t0,
                    "final_profile": profile
                }
        else:
            stable_streak = 0

    return {
        "status": "MAX_ROUNDS_REACHED",
        "rounds": max_rounds,
        "iters": atomic_steps,
        "time_sec": time.perf_counter() - t0,
        "final_profile": profile
    }

# ================================================================
# 4. Main Experiment Orchestrator and Summarizer
# ================================================================

def run_ps_existence_robustness_study(
    param_grid: List[Dict[str, Any]],
    instances_each: int = 100,
    num_restarts: int = 5,
    master_seed: int = 2025
) -> pd.DataFrame:
    """
    Orchestrates the full robustness study for PS BNE existence across a grid of parameters.
    """
    master_rng = MasterRNG(master_seed)
    results = []
    
    total_runs = len(param_grid) * instances_each
    pbar = tqdm(total=total_runs, desc="Running PS BNE Existence Study")

    for params in param_grid:
        n, p, d = params['n'], params['p'], params['d']
        
        stats = {'CONVERGED': 0, 'CYCLE_DETECTED': 0, 'MAX_ROUNDS_REACHED': 0,
                 'restart_conv': 0, 'restart_cycle': 0, 'restart_max': 0}
        
        for inst_id in range(instances_each):
            inst_rng = master_rng.spawn()
            instance = generate_random_instance(
                n, params.get('num_strategies', 2), params.get('num_resources', 3), d, inst_rng
            )
            
            found_bne_in_restarts = False
            final_status_for_instance = "MAX_ROUNDS_REACHED"
            for restart_id in range(num_restarts):
                run_rng = master_rng.spawn()
                report = run_brd_for_ps(
                    instance, p, 
                    tie_break=params.get('tie_break', "random"), 
                    noise_sigma=params.get('noise_sigma', 0.0), 
                    rng=run_rng,
                    mc_threshold_n=params.get('mc_threshold_n', 12),
                    mc_samples=params.get('mc_samples', 2000)
                )
                
                if report['status'] == 'CONVERGED':
                    stats['restart_conv'] += 1
                    found_bne_in_restarts = True
                    break
                else:
                    final_status_for_instance = report['status']
                    if report['status'] == 'CYCLE_DETECTED':
                        stats['restart_cycle'] += 1
                    else:
                        stats['restart_max'] += 1

            stats[final_status_for_instance if not found_bne_in_restarts else 'CONVERGED'] += 1
            pbar.update(1)

        results.append({**params, **stats})
        
    pbar.close()
    
    df = pd.DataFrame(results)
    for key in ['CONVERGED', 'CYCLE_DETECTED', 'MAX_ROUNDS_REACHED']:
        df[f'rate_{key.lower()}'] = df[key] / instances_each
        
    return df

# === Topology-driven PS–BRD experiment cell (single, self-contained) ===
# Requires:
# - expected_costs_ps(instance, profile, p, ...)  # your cost routine
# - run_brd_for_ps(instance, p, ...)              # your BRD routine (asynchronous GS + cycle detection)

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Literal
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Graph utilities
try:
    import networkx as nx
except ImportError:
    raise ImportError("Please install networkx: pip install networkx")

# ----------------------------
# 1) Graph generators & paths
# ----------------------------
def gen_er_graph(num_nodes: int, p_edge: float, seed: Optional[int]=None, directed: bool=False):
    """Generate an Erdős–Rényi graph and keep the giant component (undirected case)."""
    rng = np.random.default_rng(seed)
    if directed:
        G = nx.gnp_random_graph(num_nodes, p_edge, seed=int(rng.integers(1e9)), directed=True)
    else:
        G = nx.gnp_random_graph(num_nodes, p_edge, seed=int(rng.integers(1e9)))
        if not nx.is_connected(G):
            # keep giant component to ensure s-t paths exist
            G = G.subgraph(max(nx.connected_components(G), key=len)).copy()
    return G

def gen_grid_graph(width: int, height: int):
    """Generate an undirected grid and relabel nodes to 0..N-1."""
    G = nx.grid_2d_graph(width, height)
    mapping = {node: i for i, node in enumerate(G.nodes())}
    return nx.relabel_nodes(G, mapping)

def gen_tree_graph(num_nodes: int, seed: Optional[int]=None):
    """Generate a random tree (uniform)."""
    rng = np.random.default_rng(seed)
    return nx.random_tree(n=num_nodes, seed=int(rng.integers(1e9)))

def gen_series_parallel(seed: Optional[int]=None, depth: int=3):
    """Generate a tiny series–parallel directed graph; return (G,s,t)."""
    rng = np.random.default_rng(seed)
    G = nx.DiGraph()
    s, t = 0, 1
    G.add_edge(s, t)
    next_node = 2
    for _ in range(depth):
        edges = list(G.edges())
        if not edges:
            break
        u, v = edges[int(rng.integers(len(edges)))]
        if rng.random() < 0.5:
            # parallel via two chains
            mid1 = next_node; next_node += 1
            mid2 = next_node; next_node += 1
            G.remove_edge(u, v)
            G.add_edge(u, mid1); G.add_edge(mid1, v)
            G.add_edge(u, mid2); G.add_edge(mid2, v)
        else:
            # series subdiv
            m = next_node; next_node += 1
            G.remove_edge(u, v)
            G.add_edge(u, m); G.add_edge(m, v)
    # choose source/sink heuristically
    indeg = G.in_degree()
    outdeg = G.out_degree()
    sources = [n for n, d in indeg if d == 0] or [0]
    sinks   = [n for n, d in outdeg if d == 0] or [max(G.nodes())]
    return G, sources[0], sinks[0]

def k_simple_paths(G, s, t, k=3, maxlen: Optional[int]=None, seed: Optional[int]=None) -> List[Tuple[int,...]]:
    """Collect up to k simple s-t paths: shortest_simple_paths + random fallback."""
    paths: List[Tuple[int,...]] = []
    try:
        gen = nx.shortest_simple_paths(G, s, t)
        for _ in range(k):
            p = next(gen)
            if maxlen is None or len(p) <= maxlen:
                paths.append(tuple(p))
    except Exception:
        pass
    if len(paths) < k and nx.has_path(G, s, t):
        # fallback: reuse shortest path, possibly perturbed by random edge weights
        rng = np.random.default_rng(seed)
        for _ in range(5*k):
            if G.is_directed():
                H = G.copy()
            else:
                H = G.copy()
            # random weights to diversify shortest paths
            for e in H.edges():
                H.edges[e]["w"] = float(rng.random() + 1e-3)
            try:
                p = nx.shortest_path(H, s, t, weight="w")
                tup = tuple(p)
                if (maxlen is None or len(tup) <= maxlen) and tup not in paths:
                    paths.append(tup)
                    if len(paths) >= k:
                        break
            except Exception:
                continue
    uniq = []
    for p in paths:
        if p[0] == s and p[-1] == t and p not in uniq:
            uniq.append(p)
    return uniq[:k]

# ----------------------------
# 2) Instance from topology
# ----------------------------
@dataclass
class GameInstanceTopo:
    n: int
    num_resources: int
    num_strategies_per_player: int
    weights: np.ndarray
    a_coeffs: np.ndarray
    d: int
    strategies: List[List[Tuple[int, ...]]]  # each strategy is a tuple of edge-ids

def generate_topology_instance(
    n_players: int,
    G: Any, s: int, t: int,
    d: int,
    k_per_player: int = 3,
    a_low: float = 0.5, a_high: float = 1.5,
    w_low: float = 0.5, w_high: float = 2.0,
    seed: Optional[int] = None
) -> GameInstanceTopo:
    """Map graph edges to resources; each player's strategy set is K s-t paths (as edge-id sets)."""
    rng = np.random.default_rng(seed)
    # edge indexing
    edges = list(G.edges())
    edge_to_id = {e: i for i, e in enumerate(edges)}
    m = len(edges)
    # build strategies
    strategies: List[List[Tuple[int, ...]]] = []
    for _ in range(n_players):
        node_paths = k_simple_paths(G, s, t, k=k_per_player, maxlen=None, seed=int(rng.integers(1e9)))
        if not node_paths:
            raise ValueError("No s-t path found for this topology.")
        edge_paths = []
        for pth in node_paths:
            eidx = []
            for u, v in zip(pth[:-1], pth[1:]):
                if (u, v) in edge_to_id:
                    eidx.append(edge_to_id[(u, v)])
                elif (v, u) in edge_to_id:  # undirected graph case
                    eidx.append(edge_to_id[(v, u)])
                else:
                    # if edge missing, skip (shouldn't happen if pth is in G)
                    pass
            # use unique edges per strategy (set), sorted for stability
            eidx = tuple(sorted(set(eidx)))
            if eidx and eidx not in edge_paths:
                edge_paths.append(eidx)
        while len(edge_paths) < k_per_player:
            edge_paths.append(edge_paths[-1])
        strategies.append(edge_paths[:k_per_player])

    return GameInstanceTopo(
        n=n_players,
        num_resources=m,
        num_strategies_per_player=k_per_player,
        weights=rng.uniform(w_low, w_high, size=n_players),
        a_coeffs=rng.uniform(a_low, a_high, size=m),
        d=int(d),
        strategies=strategies
    )

# ----------------------------
# 3) Influence matrix (sufficient condition certificate)
# ----------------------------
def estimate_influence_matrix(
    inst: Any, p: float,
    cost_fn,
    probe_profiles: List[Tuple[int,...]],
) -> np.ndarray:
    """
    Conservative influence matrix M: M[i,j] upper-bounds how much j's unilateral change
    can increase i's expected cost (absolute increase, not normalized).
    """
    n = inst.n
    M = np.zeros((n, n), dtype=float)
    for prof in probe_profiles:
        base = cost_fn(inst, prof, p)["player_costs"]
        for j in range(n):
            k = inst.num_strategies_per_player
            # scan j's strategies; keep min & max by j's own cost as two extremes
            costs_j = []
            for sj in range(k):
                prof2 = list(prof); prof2[j] = sj
                c = cost_fn(inst, tuple(prof2), p)["player_costs"]
                costs_j.append((sj, c))
            costs_j.sort(key=lambda x: x[1][j])
            extremes = [costs_j[0][0], costs_j[-1][0]] if k >= 2 else [costs_j[0][0]]
            for sj in extremes:
                prof2 = list(prof); prof2[j] = sj
                c2 = cost_fn(inst, tuple(prof2), p)["player_costs"]
                delta = c2 - base
                for i in range(n):
                    if i == j: 
                        continue
                    M[i, j] = max(M[i, j], float(max(0.0, delta[i])))
    return M

def contraction_certificate(M: np.ndarray, norm: Literal["one","inf"]="one") -> bool:
    """Simple norm-based certificate: ||M||_1 < 1 (or ||M||_inf < 1)."""
    if norm == "one":
        return np.abs(M).sum(axis=0).max() < 1.0
    else:
        return np.abs(M).sum(axis=1).max() < 1.0

# ----------------------------
# 4) Topology builder
# ----------------------------
def build_topology(topology_cfg: Dict[str, Any], seed: Optional[int]=None):
    """Create (G, s, t, topo_label) from cfg."""
    rng = np.random.default_rng(seed)
    ttype = topology_cfg["type"]
    if ttype == "er":
        G = gen_er_graph(topology_cfg["num_nodes"], topology_cfg["p_edge"], seed=int(rng.integers(1e9)), directed=False)
        # choose far-ish s,t by eccentricity
        nodes = list(G.nodes())
        s = int(nodes[0])
        # pick t as a node with max distance from s
        lengths = nx.single_source_shortest_path_length(G, s)
        t = max(lengths, key=lengths.get)
        label = f"ER(n={G.number_of_nodes()},p={topology_cfg['p_edge']})"
        return G, s, t, label
    elif ttype == "grid":
        G = gen_grid_graph(topology_cfg["width"], topology_cfg["height"])
        s, t = 0, G.number_of_nodes()-1
        label = f"Grid({topology_cfg['width']}x{topology_cfg['height']})"
        return G, s, t, label
    elif ttype == "tree":
        G = gen_tree_graph(topology_cfg["num_nodes"], seed=int(rng.integers(1e9)))
        # choose two far leaves by diameter endpoints
        # fallback: pick random distinct nodes
        try:
            s, t = nx.diameter(G), None  # just to trigger exception for next line if not cached
        except Exception:
            pass
        # use double sweep
        u = list(G.nodes())[0]
        lengths = nx.single_source_shortest_path_length(G, u)
        s = max(lengths, key=lengths.get)
        lengths = nx.single_source_shortest_path_length(G, s)
        t = max(lengths, key=lengths.get)
        label = f"Tree(n={G.number_of_nodes()})"
        return G, s, t, label
    elif ttype == "sp":
        G, s, t = gen_series_parallel(seed=int(rng.integers(1e9)), depth=topology_cfg.get("depth", 3))
        label = f"SeriesParallel(depth={topology_cfg.get('depth',3)})"
        return G, s, t, label
    
    elif ttype == "ring":
        n_nodes = int(topology_cfg["num_nodes"])
        G = nx.cycle_graph(n_nodes)
        s = 0
        t = n_nodes // 2
        label = f"Ring(n={n_nodes})"
        return G, s, t, label
    else:
        raise ValueError(f"Unknown topology type: {ttype}")

# ----------------------------
# 5) Orchestrator
# ----------------------------
def run_ps_brd_topology_experiments(
    grid: List[Dict[str, Any]],
    instances_each: int = 50,
    restarts: int = 5,
    master_seed: int = 2026,
) -> pd.DataFrame:
    """
    For each config in `grid`, generate many topology-driven instances,
    compute a simple contraction certificate, run BRD with restarts, and aggregate stats.
    Each item in `grid` must contain:
      {
        'topology': {'type': 'er'|'grid'|'tree'|'sp', ...},
        'n': int, 'p': float, 'd': int,
        'k_per_player': int,              # strategies per player
        'mc_threshold_n': int, 'mc_samples': int,
        'tie_break': 'random'|'first',
        'noise_sigma': float,
      }
    """
    master_rng = np.random.default_rng(master_seed)
    rows = []
    total = len(grid) * instances_each
    pbar = tqdm(total=total, desc="Topology PS–BRD experiments")

    for cfg in grid:
        n, p, d = cfg["n"], cfg["p"], cfg["d"]
        topo_cfg = cfg["topology"]
        tie_break = cfg.get("tie_break", "random")
        noise_sigma = cfg.get("noise_sigma", 0.0)
        k_per_player = cfg.get("k_per_player", 3)
        mc_threshold_n = cfg.get("mc_threshold_n", 12)
        mc_samples = cfg.get("mc_samples", 2000)

        # counters and collectors
        cnt_conv = cnt_cycle = cnt_max = 0
        cert_true = 0
        rounds_conv, iters_conv, times_conv = [], [], []
        times_all = []

        for _ in range(instances_each):
            # Build topology, instance
            G, s, t, topo_label = build_topology(topo_cfg, seed=int(master_rng.integers(1e9)))
            inst = generate_topology_instance(
                n_players=n, G=G, s=s, t=t, d=d,
                k_per_player=k_per_player,
                seed=int(master_rng.integers(1e9))
            )

            # Cost fn wrapper with correct MC routing
            cost_fn = lambda inst_, prof_, p_ : expected_costs_ps(
                inst_, prof_, p_,
                mc_threshold_n=mc_threshold_n,
                mc_samples=mc_samples,
                rng=np.random.default_rng(int(master_rng.integers(1e9)))
            )

            # Build a few probe profiles to estimate influence matrix
            probe_profiles = []
            rng_prof = np.random.default_rng(int(master_rng.integers(1e9)))
            for _r in range(min(6, 2*n)):  # a handful of random profiles
                prof = tuple(int(x) for x in rng_prof.integers(0, inst.num_strategies_per_player, size=inst.n))
                probe_profiles.append(prof)
            # Add "all-zeros" and "all-ones" if available
            probe_profiles.append(tuple(0 for _ in range(inst.n)))
            if inst.num_strategies_per_player > 1:
                probe_profiles.append(tuple(1 for _ in range(inst.n)))

            M = estimate_influence_matrix(inst, p, cost_fn, probe_profiles)
            if contraction_certificate(M, norm="one"):
                cert_true += 1

            # Multi-restart BRD
            found = False
            last_report = None
            for _rs in range(restarts):
                run_rng = np.random.default_rng(int(master_rng.integers(1e9)))
                rep = run_brd_for_ps(
                    inst, p,
                    tie_break=tie_break,
                    noise_sigma=noise_sigma,
                    rng=run_rng,
                    mc_threshold_n=mc_threshold_n,
                    mc_samples=mc_samples
                )
                times_all.append(rep["time_sec"])
                last_report = rep
                if rep["status"] == "CONVERGED":
                    found = True
                    rounds_conv.append(rep.get("rounds", np.nan))
                    iters_conv.append(rep.get("iters", np.nan))
                    times_conv.append(rep["time_sec"])
                    break

            if found:
                cnt_conv += 1
            else:
                if last_report["status"] == "CYCLE_DETECTED":
                    cnt_cycle += 1
                else:
                    cnt_max += 1

            pbar.update(1)

        # aggregate
        runs = instances_each
        def _binom_ci(k, n, z=1.96):
            if n == 0: 
                return np.nan, np.nan, np.nan
            ph = k / n
            se = np.sqrt(ph * (1 - ph) / n)
            return ph, max(0.0, ph - z*se), min(1.0, ph + z*se)

        r_conv, lo, hi = _binom_ci(cnt_conv, runs)

        rows.append({
            "topology": topo_label,
            "n": n, "p": p, "d": d, "k_per_player": k_per_player,
            "tie_break": tie_break, "noise_sigma": noise_sigma,
            "mc_threshold_n": mc_threshold_n, "mc_samples": mc_samples,
            "runs": runs,
            "conv": cnt_conv, "cycle": cnt_cycle, "max_rounds": cnt_max,
            "converge_rate": r_conv, "converge_rate_ci_low": lo, "converge_rate_ci_high": hi,
            "cert_true": cert_true, "cert_rate": cert_true / runs,
            "median_rounds_conv": float(np.median(rounds_conv)) if rounds_conv else np.nan,
            "p95_rounds_conv": float(np.quantile(rounds_conv, 0.95)) if rounds_conv else np.nan,
            "median_iters_conv": float(np.median(iters_conv)) if iters_conv else np.nan,
            "p95_iters_conv": float(np.quantile(iters_conv, 0.95)) if iters_conv else np.nan,
            "median_time_all": float(np.median(times_all)) if times_all else np.nan,
            "p95_time_all": float(np.quantile(times_all, 0.95)) if times_all else np.nan,
            "median_time_conv": float(np.median(times_conv)) if times_conv else np.nan,
            "p95_time_conv": float(np.quantile(times_conv, 0.95)) if times_conv else np.nan,
        })

    pbar.close()
    return pd.DataFrame(rows)


In [30]:

# ----------------------------
# 6) Quick demo grid (small)
# ----------------------------
demo_grid = [
    # ER: light overlap (should often converge)
    {"topology": {"type":"er","num_nodes":12,"p_edge":0.15},
     "n": 2, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},
    # ER: denser (more overlap, more cycles risk)
    {"topology": {"type":"er","num_nodes":15,"p_edge":0.35},
     "n": 2, "p": 0.9, "d": 4, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 1e-4},
    # Grid: structured overlap
    {"topology": {"type":"grid","width":3,"height":4},
     "n": 2, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "first", "noise_sigma": 0.0},
    # Series–Parallel: often easier (near-tree)
    {"topology": {"type":"sp","depth":3},
     "n": 2, "p": 0.5, "d": 4, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},
]

# Run a tiny pilot (adjust instances_each/restarts upward for paper-scale runs)
df_topo = run_ps_brd_topology_experiments(
    grid=demo_grid,
    instances_each=20,   # start small to validate pipeline
    restarts=5,
    master_seed=2027
)

display(df_topo)


Topology PS–BRD experiments:   0%|          | 0/80 [00:00<?, ?it/s]

,topology,n,p,d,k_per_player,tie_break,noise_sigma,mc_threshold_n,mc_samples,runs,...,cert_true,cert_rate,median_rounds_conv,p95_rounds_conv,median_iters_conv,p95_iters_conv,median_time_all,p95_time_all,median_time_conv,p95_time_conv
0,"ER(n=7,p=0.15)",2,0.5,3,3,random,0.0000,12,2000,20,...,4,0.20,4.0,4.0,8.0,8.0,0.312227,0.603156,0.468139,0.576937
1,"ER(n=15,p=0.35)",2,0.9,4,3,random,0.0001,12,2000,20,...,0,0.00,4.0,4.0,8.0,8.0,0.897760,1.038008,0.897760,1.038008
2,Grid(3x4),2,0.5,3,3,first,0.0000,12,2000,20,...,1,0.05,4.0,4.0,8.0,8.0,0.607420,0.646374,0.607420,0.646374
3,SeriesParallel(depth=3),2,0.5,4,3,random,0.0000,12,2000,20,...,2,0.10,4.0,4.5,8.0,9.0,0.292776,0.425359,0.344172,0.439623


In [31]:

extended_grid = [

    {"topology": {"type": "er", "num_nodes": 12, "p_edge": 0.2},
     "n": 2, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},

    {"topology": {"type": "grid", "width": 4, "height": 3},
     "n": 2, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},


    {"topology": {"type": "er", "num_nodes": 15, "p_edge": 0.25},
     "n": 2, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},

    {"topology": {"type": "grid", "width": 5, "height": 5},
     "n": 2, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},


    {"topology": {"type": "er", "num_nodes": 12, "p_edge": 0.2},
     "n": 3, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},

    {"topology": {"type": "grid", "width": 4, "height": 3},
     "n": 3, "p": 0.5, "d": 3, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},
]


df_extended = run_ps_brd_topology_experiments(
    grid=extended_grid,
    instances_each=2000,   
    restarts=10,           
    master_seed=2030       
)

# result
display(df_extended[['topology', 'n', 'p', 'd', 'converge_rate', 'cycle', 'median_iters_conv']])

Topology PS–BRD experiments:   0%|          | 0/12000 [00:00<?, ?it/s]

,topology,n,p,d,converge_rate,cycle,median_iters_conv
0,"ER(n=11,p=0.2)",2,0.5,3,0.8520,296,8.0
1,Grid(4x3),2,0.5,3,0.9990,2,8.0
2,"ER(n=15,p=0.25)",2,0.5,3,0.9915,17,8.0
3,Grid(5x5),2,0.5,3,0.9995,1,8.0
4,"ER(n=11,p=0.2)",3,0.5,3,0.8110,378,12.0
5,Grid(4x3),3,0.5,3,0.9990,2,12.0


In [32]:
df_extended.to_csv('converge_results.csv')

In [39]:
ring_grid_d4= [
    {"topology": {"type": "er", "num_nodes": 12, "p_edge": 0.2},
     "n": 2, "p": 0.5, "d": 4, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},
    {"topology": {"type": "grid", "width": 5, "height": 5},
     "n": 2, "p": 0.5, "d": 4, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},
    {"topology": {"type": "grid", "width": 4, "height": 3},
     "n": 2, "p": 0.5, "d": 4, "k_per_player": 3,
     "mc_threshold_n": 12, "mc_samples": 2000,
     "tie_break": "random", "noise_sigma": 0.0},

    
]

df_ring_grid_d4 = run_ps_brd_topology_experiments(
    grid=ring_grid_d4,
    instances_each=2000,   
    restarts=10,          
    master_seed=2043
)

# 展示关键指标
display(df_ring_grid_d4[[
    "topology", "n", "p", "d",
    "converge_rate", "cycle",
    "median_iters_conv",
    "median_time_all", "p95_time_all",
    "median_time_conv", "p95_time_conv"
]])

Topology PS–BRD experiments:   0%|          | 0/6000 [00:00<?, ?it/s]

,topology,n,p,d,converge_rate,cycle,median_iters_conv,median_time_all,p95_time_all,median_time_conv,p95_time_conv
0,"ER(n=12,p=0.2)",2,0.5,4,0.852,296,8.0,0.368939,0.687404,0.457632,0.680524
1,Grid(5x5),2,0.5,4,1.000,0,8.0,1.052413,1.167539,1.053332,1.159705
2,Grid(4x3),2,0.5,4,1.000,0,8.0,0.584918,0.634797,0.585116,0.634052


In [40]:
df_ring_grid_d4.to_csv('converge_results_final.csv')